In [10]:
# Initialize Otter
import otter
grader = otter.Notebook("proj1.ipynb")

# Project 1: Cal EnviroScreen

**Developers**: Bing Concepcion, James Geronimo, Zcjanin Ollesca

---

In this project, students will gain experience with fundamental Exploratory Data Analysis? using the CalEnviroScreen data. This project will build on methods introduced in lab. It will also serve as an application of data science in the field of social sciences and **environmental justice**. According to state law, environmental justice refers to the "fair treatment of people of all races, cultures, and incomes with respect to the development, adoption, implementation and enforcement of environmental laws, regulations, and policies." 

By the end of this project, students will be able to:
- Perform basic tabular analysis using pandas and interpret results
- Extract data using conditional selection
- Visualize and analyze CalEnviroScreen data
- Identify how data-driven decision making can guide policy and resource allocation

## Table of Contents

1. [Introduction](#1.-introduction)
2. [Exploratory Data Analysis](#exploratory-data-analysis)
3. [Visualizing the Data](#visualizing-the-data)
4. [Data-Driven Decision Making](#data-driven-decision-making)

### Import Modules

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "svg"
pd.set_option("display.max_columns", None)

---

## 1. Introduction


The [California Communities Environmental Health Screening Tool](https://oehha.ca.gov/calenviroscreen) (CalEnviroScreen) provides accessible demographic and environmental information to identify communities that are susceptible to certain types of pollution. This tool utilizes environmental, health, and socioeconomic information to produce scores for every census tract in California, allowing us to compare qualities of different communities. 

### 1.1 Reading in CalEnviroScreen Data

To begin exploring CalEnviroScreen, run the following cell to read in the data.

In [12]:
# Read in the data
ces = pd.read_csv('modified_cal_enviro_screen.csv')
ces.head()

,Census Tract,Total Population,California County,ZIP,Approximate Location,Longitude,Latitude,CES 4.0 Score,CES 4.0 Percentile,CES 4.0 Percentile Range,Ozone,Ozone Pctl,PM2.5,PM2.5 Pctl,Diesel PM,Diesel PM Pctl,Drinking Water,Drinking Water Pctl,Lead,Lead Pctl,Pesticides,Pesticides Pctl,Tox. Release,Tox. Release Pctl,Traffic,Traffic Pctl,Cleanup Sites,Cleanup Sites Pctl,Groundwater Threats,Groundwater Threats Pctl,Haz. Waste,Haz. Waste Pctl,Imp. Water Bodies,Imp. Water Bodies Pctl,Solid Waste,Solid Waste Pctl,Pollution Burden,Pollution Burden Score,Pollution Burden Pctl,Asthma,Asthma Pctl,Low Birth Weight,Low Birth Weight Pctl,Cardiovascular Disease,Cardiovascular Disease Pctl,Education,Education Pctl,Linguistic Isolation,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,-2780,Fresno,93706,Fresno,-119.781696,36.709695,93.18,120.00,95-100% (highest scores),0.060,82.48,13.906348,97.71,1.123,98.72,733.95,84.39,89.60,96.47,1.00,42.90,4859.094604,92.20,1037.095744,60.38,70.50,98.19,54.25,91.17,3.10,96.31,0,0.00,6.00,79.95,78.98,9.64,99.93,129.54,97.22,7.80,95.62,21.47,92.25,44.5,93.23,16.0,79.37,76.0,98.92,12.8,93.83,30.3,91.04,93.16,9.66,99.72
1,6077000700,-4680,San Joaquin,95206,Stockton,-121.287873,37.943173,86.65,119.99,95-100% (highest scores),0.046,44.98,11.884085,72.59,0.538,91.15,389.85,41.55,77.30,86.83,63.13,73.66,519.628001,52.35,856.395935,48.29,61.90,97.46,78.60,95.09,1.27,88.58,13,91.87,9.25,89.28,73.43,8.97,99.29,105.88,94.22,6.88,88.72,20.26,88.12,46.4,94.47,29.7,95.53,73.2,98.39,19.8,99.21,31.2,92.28,93.17,9.66,99.74
2,6037204920,-2751,Los Angeles,90023,Los Angeles,-118.197497,34.017500,82.39,119.97,95-100% (highest scores),0.048,53.73,12.251640,89.21,0.781,96.55,787.94,92.53,92.56,98.40,0.00,0.00,3682.693278,87.73,2522.622269,92.84,38.75,92.96,20.50,68.92,11.62,99.66,7,66.74,4.85,73.08,77.67,9.48,99.86,76.10,82.76,7.11,90.89,20.87,90.15,52.2,97.39,17.1,81.55,62.6,93.39,6.4,61.53,20.3,63.97,83.75,8.69,95.79
3,6019000700,-3664,Fresno,93706,Fresno,-119.827707,36.734535,81.33,119.96,95-100% (highest scores),0.060,82.48,13.520939,95.88,0.174,57.09,733.95,84.39,68.39,77.00,44.57,71.59,1630.342707,74.93,690.502159,35.28,16.50,77.32,9.50,44.83,2.36,94.12,0,0.00,5.75,78.14,67.85,8.28,97.40,139.45,98.24,10.65,99.78,22.68,94.57,41.4,90.94,15.7,78.71,65.7,95.35,15.7,97.35,35.4,96.41,94.64,9.82,99.89
4,6019000200,-2689,Fresno,93706,Fresno,-119.805504,36.735491,80.75,119.95,95-100% (highest scores),0.060,82.48,13.818959,97.47,1.390,99.30,733.95,84.39,75.41,85.12,16.63,64.41,1975.207988,78.96,909.650882,52.26,10.50,62.45,28.25,78.05,0.35,56.40,0,0.00,0.00,0.00,66.83,8.16,96.94,139.08,98.17,10.25,99.67,22.64,94.43,43.6,92.57,20.0,86.56,72.7,98.30,13.7,95.29,32.7,94.16,95.40,9.90,99.95


Before we begin any tabular analysis, let's familiarize ourselves with this DataFrame by looking at its dimensions and the data types of its columns. Run the following code cell to print the dimensions of `ces`.

### The Granularity of our Data

**Granularity** refers to the *level of detail* captured within a dataset. It defines the **smallest unit of observation**, or  *how* specific the data points are. 

* A dataset with *high granularity* contains fine-grained, detailed records such as individual transactions and sensor readings.
* A dataset with *low granularity* consists of more aggregated information such as monthly sales summaries and regional statistics.

Each row in the dataset represents a single census tract in California. We can think of a census tract as a small, relatively permanent statistical subdivision of a county, defined by the [U.S. Census Bureau](https://www.census.gov/).

The data we have available to us provides environmental and demographic data at this census tract level, including pollution burden indicators (e.g., air quality, hazardous waste sites), as well as some population characteristics (e.g., asthma rates, linguistic isolation, poverty levels). 

In total, California has around 8,057 census tracts. Let's examine how many we have in our dataset: 

In [13]:
# Run this cell
cal_enviro_screen_shape = ces.shape
num_rows = cal_enviro_screen_shape[0]
num_columns = cal_enviro_screen_shape[1]

print(f"Our version of the CalEnviroScreen data has {num_rows} rows and {num_columns} columns")

Our version of the CalEnviroScreen data has 8035 rows and 58 columns


Let's take a look and see if all these rows are unique census tracts: 

In [14]:
number_unique = ces["Census Tract"].nunique()
print(f"There are {number_unique} unique census tracts in our dataset!")

There are 8035 unique census tracts in our dataset!


Great! Our dataset contains 8,305 unique census tracts, matching the total number of rows. This confirms that there are no duplicate entries

**Question 1.1.**: Relative to county or state-level data, how would you describe the granularity of the CalEnviroScreen dataset? Assign `relative_granularity` to an integer corresponding to the assumption.

1. Lower granularity, because it aggregates data across multiple regions.
2. Higher granularity, because it provides more detailed, localized data.
3. The same level of granularity as county and state-level data.
4. Granularity cannot be determined without individual-level data.

In [23]:
relative_granularity = 2

In [24]:
grader.check("q11")

q11 results: All test cases passed!

### Properties of Our Data

Now, let's take a look at some of the columns of our DataFrame. We can do so using the `.columns` attribute of DataFrame objects:

In [25]:
# Run this cell
print(ces.columns)

Index(['Census Tract', 'Total Population', 'California County', 'ZIP',
       'Approximate Location', 'Longitude', 'Latitude', 'CES 4.0 Score',
       'CES 4.0 Percentile', 'CES 4.0 Percentile Range', 'Ozone', 'Ozone Pctl',
       'PM2.5', 'PM2.5 Pctl', 'Diesel PM', 'Diesel PM Pctl', 'Drinking Water',
       'Drinking Water Pctl', 'Lead', 'Lead Pctl', 'Pesticides',
       'Pesticides Pctl', 'Tox. Release', 'Tox. Release Pctl', 'Traffic',
       'Traffic Pctl', 'Cleanup Sites', 'Cleanup Sites Pctl',
       'Groundwater Threats', 'Groundwater Threats Pctl', 'Haz. Waste',
       'Haz. Waste Pctl', 'Imp. Water Bodies', 'Imp. Water Bodies Pctl',
       'Solid Waste', 'Solid Waste Pctl', 'Pollution Burden',
       'Pollution Burden Score', 'Pollution Burden Pctl', 'Asthma',
       'Asthma Pctl', 'Low Birth Weight', 'Low Birth Weight Pctl',
       'Cardiovascular Disease', 'Cardiovascular Disease Pctl', 'Education',
       'Education Pctl', 'Linguistic Isolation', 'Linguistic Isolation Pctl',

Let's also examine the data types of some of our columns

In [26]:
# Run this cell to return the data type of the first 10 columns of the DataFrame
ces.dtypes[:10]

Census Tract                  int64
Total Population              int64
California County            object
ZIP                           int64
Approximate Location         object
Longitude                   float64
Latitude                    float64
CES 4.0 Score               float64
CES 4.0 Percentile          float64
CES 4.0 Percentile Range     object
dtype: object

### Indicators

The Cal Enviroscreen [Indicators Overview](https://oehha.ca.gov/calenviroscreen/indicators-overview) defines an indicator as a measure of environmental and population characteristics, which can then be used to assess the vulnerability of pollution for a specific country. These are grouped into four categories:

* Exposure Indicators – Measure pollution levels that people may come into contact with.
* Environmental Effects Indicators – Identify toxic chemical sites near communities.
* Sensitive Population Indicators – Reflect health conditions or age factors that make individuals more vulnerable to pollution.
* Socioeconomic Factor Indicators – Represent social and economic conditions that can increase stress and make communities more susceptible to pollution’s effects.

Each indicator is scored separately, contributing to the overall CalEnviroScreen score.

**Question 1.2.** Select *one* specific indicator from each group and provide a brief description of what it represents, what it measures, and what it's purpose is.  

For a full list of these indicators, visit the CalEnviroScreen [Indicators Overview](https://oehha.ca.gov/calenviroscreen/indicators-overview) and scroll to the bottom of the page.

<!-- BEGIN QUESTION -->

**Question 1.2.1.** Find an Exposure Indicator and write a brief description as outlined above.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 1.2.2.** Find an Environmental Effects Indicator and write a brief description as outlined above.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 1.2.3.** Find an Exposure Indicator and write a brief description as outlined above.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 1.2.4.** Find a Socioeconomic Factor Indicator and write a brief description as outlined above.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### Reading in Community College Data

Let's also read in data containing information about the locations of community colleges in California. We will familiarize ourselves with this data the same way we did for `ces`.

In [ ]:
# Read in the data
colleges = pd.read_excel("colleges.xlsx")
colleges.head()

<!-- BEGIN QUESTION -->

**Question 1.3.** What is the granularity of the `colleges` dataset? What does each row represent? 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 1.4.** What are the dimensions of the `colleges` dataset? Fill in the code cell with the necessary code and print your answer.

Hint: We did this in a previous cell! 

In [ ]:
community_college_shape = ...
cc_num_rows = ...
cc_num_columns = ...

print(f"The Community College dataset has {cc_num_rows} rows and {cc_num_columns} columns")

In [ ]:
grader.check("q22")

**Question 1.5.** What are the columns of the dataset? List them all below

In [ ]:
college_columns = ...
college_columns

In [ ]:
grader.check("q15")

### Merging `ces` and `colleges`

Now that we have an understanding of both of our DataFrames, we will merge these two DataFrames to analyze the socioeconomic conditions of various community colleges in California. We can make use of the [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function in pandas for this. 

To use `pd.merge()`, we will pass in the following parameters into the function respectively:
- The first DataFrame being merged
- The second DataFrame being merged
- `how` indicates the type of merge to be used
- `left_on` and `right_on` parameters are assigned to the string names of the columns to be used when performing the join. These two on parameters tell pandas what values should act as pairing keys to determine which rows to merge across the DataFrames. We’ll talk more about this idea of a pairing key next lecture.

We will be using an *inner* merge which will use the intersection of keys from both DataFrames( similar to a SQL inner join). It will preserve the order of the left keys.

**Question 1.6.** Merge the `ces` and `colleges` dataset

*Hint: Feel free to reference the Pandas [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)*

In [ ]:
ces_cc = ...
ces_cc.shape

In [ ]:
grader.check("q16")

---

## 2. Exploratory Data Analysis

In this next section, we will focus on tabular analysis of the DataFrames that we have constructed. Specifically we will practice utility functions, slicing, conditional selection, and aggregations.

The attribute, `CES 4.0 Score`, reveals the CalEnviroScreen Score for a given census tract.  Essentially, this score takes into account the `Pollution Burden`, which accounts for factors like air and water quality, and `Population Characteristics`, which assesses the sensitivity of the population to pollution, to measure the socioeconomic conditions across various communities. Higher scores indicate greater cumulative environmental and health burdens. This score aids state agencies, local governments, and community organizations identify communities that are more susceptible to environmental health concerns.

### 2.1 Utility Functions

<!-- BEGIN QUESTION -->

**Question 2.1.1** Let's take a look again at the `ces_cc` dataset, particularily the values in the `CES 4.0 Percentile` column. What do we notice is strange about these values? Feel free to run the cell below to check. 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 2.1.2** Inspect the `CES 4.0 Percentile` column in the ces_cc DataFrame. Identify its minimum and maximum values. Based on these values, determine an appropriate horizontal shift to apply to the entire column. Assign the variable `shift` to the value you would add to each entry in order to adjust the data accordingly.

* For example, if we want to subtract 5 from every value, set shift = -5.

* If we want to add 5, set shift = 5.

In [ ]:
max_ces_percentile = ...
min_ces_percentile = ...
shift = ...
print(f"We want to shift our values by {shift}")

In [ ]:
grader.check("q212")

**Question 2.1.3** Using your value of `shift` from the previous question, adjust the values in the `CES 4.0 Percentile` column using pandas. Assign the resulting dataframe to `adjusted_ces_cc`

In [ ]:
adjusted_ces_cc = ...
adjusted_ces_cc = ...
adjusted_ces_cc.head(5)

In [ ]:
grader.check("q213")

**Question 2.1.4** Oh no! It looks like all the values in the `Total Population` column of the `adjusted_ces_cc` DataFrame are negative. We've been told these numbers were mistakenly recorded as negative, and they should actually be positive.

Create a new DataFrame called `pop_adjusted_ces_cc` that is a copy of `adjusted_ces_cc`, but with all values in the `Total Population` column converted to their positive equivalents.

In [ ]:
pop_adjusted_ces_cc = ...
pop_adjusted_ces_cc.head(5)

In [ ]:
grader.check("q214")

Now, we will reassign the `ces_cc` variable to our *cleaned* DataFrame in the form of `pop_adjusted_ces_cc`

Now that we've cleaned up our data, let's go about finding the **most** polluted census tracts and show which colleges are impacted.

**Question 2.1.5** Write code to find the top 10 most polluted census tracts and display the colleges using the `CES 4.0 Score` column and the `.sort_values()` function in pandas. Assign it to the variable `most_polluted`. 

If you are feeling stuck, consult the pandas documentation for the function to understand how to use it. Be sure to display the results in *descending* order

In [ ]:
most_polluted = ...
most_polluted

In [ ]:
grader.check("q215")

<!-- BEGIN QUESTION -->

**Question 2.1.6** We notice that there are two instances of Compton Community College and Fresno Pacific University. Provide a brief explanation for why this could be the case? Feel free to reference the `ces_cc` table as reference. 

_Type your answer here, replacing this text._

<!-- END QUESTION -->


### 2.2: A special case with PM2.5

Produced from vehicle emissions, industrial pollution, and wildfires, PM2.5 (Particulate Matter (PM) that is 2.5 micrometers or smaller in diameter) is one of the most dangerous air pollutants. Their tiny size allows them to penetrate deep into the lungs and even enter the bloodstream. Exposure to PM2.5 is linked to respiratory diseases (like asthma and bronchitis), as well as cardiovascular diseases. 

We typically measured PM2.5 in micrograms per cubic meter (µg/m³) of air. Below is a table showing the potential risks of different `PM2.5` values

|PM2.5 (µg/m³) | Air Quality Index (AQI) | Health Concern|
|---------------|------------------------|---------------|
|0 – 12	|          Good	                |Minimal impact
|12 – 35.4 |	Moderate |	Unhealthy for sensitive groups|
|35.5 – 55.4	|Unhealthy	|Risk for everyone|
|55.5 – 150	|Very Unhealthy|	Health warning issued|
|150+|	Hazardous|	Emergency conditions|


**Question 2.1.2:** Sort the dataset by `PM2.5` concentration (in descending order) and assign the resulting `DataFrame` to the variable `sorted_PM2_5`.

*Hint: You may find the `ascending=` parameter of the `.sort_values()` function useful*

In [ ]:
# SOLUTION
sorted_PM2_5 = ces_cc.sort_values(by='PM2.5', ascending=False)
sorted_PM2_5.head(3)

**Question 2.1.3:** Now that we have sorted the `ces` DataFrame by the values in the `PM2.5`, let's now go about **accessing** the highest value! Assign `highest_PM2_5` to the highest `PM2.5` concentration by using the `sorted_PM2_5` Dataframe

Make sure you use Pandas code to do this! Don't assign `highest_PM2_5` to just the highest value. 

In [ ]:
# SOLUTION
highest_PM2_5 = sorted_PM2_5["PM2.5"].iloc[0]
highest_PM2_5

**Question 2.1.4:** What *Approximate Location* and *county* does this `highest_PM2_5` value come from? Assign these values to `approx_loc_PM2_5` and `county_PM2_5` respectively

In [ ]:
# SOLUTION

approx_loc_PM2_5 = sorted_PM2_5["Approximate Location"].iloc[0]
county_PM2_5 = sorted_PM2_5["California County"].iloc[0]

approx_loc_PM2_5, county_PM2_5

**Question 2.1.5:** Based on your answers above, how would you classify the AQI for this Approximate Location? Furthermore, provide 1 reason why you think the `PM2.5` value is as high as it is for this location.

*SOLUTION: Bakersfield has a PM2.5 score of 15.788, making it a moderate AQI region.*

--- 

**Question 2.1.3:** Take a look at the difference in cities between the most polluted and least polluted census tracts. What do you notice? Are there any characteristics that cities in the most polluted census tracts have in common? What about in the least polluted census tracts? Write 2-3 sentences noting your observations. 

*YOUR ANSWER HERE...*

### 2.2 Filtering and Conditional Selection

Let's take a closer look at the census tract for El Camino College. The relevant tract number is **6037603702**. We will be using filtering methods you have learned in lab to find information on El Camino College.

**Question 2.2.1:** Filter the dataset using `.loc()` for this tract number and save the DataFrame by assigning it to the variable `ecc` using the code cell below.

In [ ]:
# SOLUTION
tract_number = 6037603702
ecc = ces_cc.loc[ces_cc['Census Tract'] == tract_number]
ecc

**Question 2.2.2:** Based on this filtered data, let's examine three new measures of environmental health and interpret the scores for El Camino College. Before writing any code let's establish what these measures are. Refer back to the CalEnviroScreen website for context on these three health measures in the data and write definitions for each. 

- `Pesticides Pctl`: *YOUR ANSWER HERE...*

- `asthma_pctl`: *YOUR ANSWER HERE...*
- `CES 4.0 Percentile`: *YOUR ANSWER HERE...*

Now, write code in the cell below to obtain the values for these metrics, as well as `Pollution Burden Pctl`, for the **El Camino College** census tract. Please use the `.iloc[]` method to filter the DataFrame. 

In [ ]:
# TODO: Fill in the ellipses

pest_pctl = ...
asthma_pctl = ...
ces4_pctl = ...
plltn_burden_pctl = ...

In [ ]:
# TODO: Fill in the ellipses

pm25_pctl = ecc["Pesticides Pctl"].iloc[0]
asthma_pctl = ecc["Asthma Pctl"].iloc[0]
ces4_pctl = ecc["CES 4.0 Percentile"].iloc[0]
plltn_burden_pctl = ecc["Pollution Burden Pctl"].iloc[0]

print(f"Pesticides Pctl: {pm25_pctl} \nAsthma Pctl: {asthma_pctl} \nCES 4.0 Percentile {ces4_pctl} \nPollution Burden Pctl {plltn_burden_pctl}")

**Question 2.2.2.** Briefly describe the real-world implications of at least two of these scores.

*SOLUTION: Any reasonable answer will receive credit*

Let's compare these four measures across other census tracts in **Los Angeles**. 

**Question 2.2.3:** Using the `ces_cc` dataset, write a line of code to filter the  dataset to contain only Los Angeles county data by using the `City` column. Additionally, make sure only the following columns are included in this new dataset: 
* `Approximate Location`

* `PM2.5 Pctl`

* ` CES 4.0 Percentile`
* `Asthma Pctl`
* `Pollution Burden Pctl`

You can use any form of conditional selection for this question (.loc(), boolean conditioning).

*Hint: Make sure to pay attention to how cities in the `City` column are formatted. Do you notice anything interesting?*

In [ ]:
# TODO: Filter for Los Angeles data

la = ces_cc[ces_cc["City"] == 'LOS ANGELES'][["Approximate Location", "PM2.5 Pctl", "Asthma Pctl", "Pollution Burden Pctl"]]
la.head(3)

**INSERT SOME KIND OF IMPUTATION QUESTION**

### 2.3 Groupby

Finally, we are going to use a popular aggregation method you have used in lab, `.groupby()`. As data scientists, we often wish to investigate trends across a larger subset of our data. For example, we may want to compute some summary statistic (the mean, median, sum, etc.) for a group of rows in our DataFrame.  Our goal is to group together rows that fall under the same category and perform an operation that aggregates across all rows in the category. Before you jump into aggregating the data, follow along with the example below for a refresher on the `.groupby()` method.

**Demo:** In this example, our goal will be to create a DataFrame that finds the mean `CES 4.0` score by unique `California County`. This demo is adapted from the [Data 100 Textbook](https://ds100.org/course-notes/pandas_3/pandas_3.html#aggregating-data-with-.groupby).

The first step to achieve this goal is to call `.groupby()` on the `ces_cc` DataFrame and passing in the necessary column. In this specific case, we want to make groups according to each unique county so we will pass in `California County`. 

In [ ]:
# DEMO: Step 1

ces_cc.groupby('California County')

Calling `.groupby()` alone will not return a viable DataFrame. This returns a `GroupBy` object, which you can imagine as a set of “mini” sub-DataFrames, where each subframe contains all of the rows from `ces_cc` that correspond to a particular county. To actually manipulate values within these “mini” DataFrames, we’ll need to call an *aggregation* method. This is a method that tells `Pandas` how to aggregate the values within the GroupBy object. Once the aggregation is applied, pandas will return a normal (now grouped) DataFrame.

One way to call an aggregation method is to call `.agg()` on your GroupBy object. You must specifiy the aggregation function you want to use by passing it into the method call as a numpy function (np.mean, np.sum) or as a string ('mean', 'median', 'size'). In our case, we want to find the mean within each county. 

There are many other aggregation functions we can use:
- `.agg("sum")`
- `.agg("max")`
- `.agg("min")`
- `.agg("mean")`
- `.agg("first")`
- `.agg("last")`

For more information, refer to the `.groupby()` [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)

In [ ]:
# DEMO: Step 2

ces_cc.groupby('California County').mean(numeric_only=True).reset_index()

Wow! That is a lot of information, and it might be a bit hard to identify which parts are useful. 

As you can see, our index now becomes the unique county names in the DataFrame and each value is the mean value for that column attribute in each county. We only want to look at the mean `CES 4.0 SCORE` score by county so we will use **double bracket notation** to filter the DataFrame for only this column. While we can perform this step after calling `.agg()`, we can also perform it after calling `.groupby()` to avoid applying the aggregation function on *all* columns of the data. This is good practice when you know which columns you want summary statistics for.

In [ ]:
# DEMO: Step 3
ces_cc.groupby('California County')[['CES 4.0 Score']].agg('mean')

Now we have a new DataFrame that contains information about the mean CES 4.0 score by county!

*Bonus Step:* If we want to look at the counties with the highest scores, We can also sort this data by using `.sort_values()`.

In [ ]:
# DEMO: Step 4 (optional)

ces_cc.groupby('California County')[['CES 4.0 Score']].agg('mean').sort_values('CES 4.0 Score', ascending=False)

Now you are ready to try out the `groupby()` function in the next question!

Let's take a look at some of the unique values in the `California County` column. Do we notice anything interesting about some of these values?

Hint: Try looking at the second and third values below and see if there's anything wrong! 



In [ ]:
ces_cc['California County'].unique()

**Question 2.3.1** Think of a way to use the `.str.strip()` method to fix the column labels! 

In [ ]:
ces_cc['California County'] = ces_cc['California County'].str.strip()

Let's verify that our unique values *did* indeed change! 

In [ ]:
ces_cc['California County'].unique()

**Question 2.3.2:** With the `la` DataFrame, find the mean value for each of these four metrics (`CES 4.0 Score`, `PM2.5`, `Asthma`, and `Pollution Burden Score`) based on the unique `City`. Assign this new dataset to a new DataFrame. 

In [ ]:
la

In [ ]:
# TODO: Write code to find the mean of PM2.5, CES4.0, Asthma, and Pollution Burden percentiles by city

city_means = ...
city_means

**Question 2.3.3:** Now instead of the mean, return the *median* value for each of the same four metrics based on the unique city.

In [ ]:
# TODO: Write code to find the median of PM2.5, CES4.0, Asthma, and Pollution Burden percentiles by city

city_medians = ...

We can also make use of custom aggregation functions using `.groupby()`. To do so, let's create a custom function called `calc_IQR` returns the Interquartile Range of a set of values

**Question 2.3.4:** Fill in the code for the function `calc_IQR` such that it calculates the Interquartile Range of a sequence of values. You can assume that the input to this function will be a Series object. 

In [ ]:
#SOLUTION
def calc_IQR(sequence): 
    return np.percentile(sequence, 75) - np.percentile(sequence, 25)


**Question 2.3.5:** Now, use the function to find the `IQR range` of `CES 4.0 Score` and output that in the table and sort these IQR values in descending order. Call this table `CES_IQR`

In [ ]:
CES_IQR = ces_cc.groupby('California County')[['CES 4.0 Score']].agg(calc_IQR).sort_values("CES 4.0 Score", ascending= False).rename(columns={'CES 4.0 Score': 'IQR diference in CES 4.0 Score'})
CES_IQR

**Question 2.3.6:** Looking at the bottom two values, we notice that the counties Siskiyou and Imperial have an IQR difference of 0. Why do you think this could be? Explain your reasoning.

*Type your answer here*

---

## 3. Visualizing the Data

In conducting exploratory data analysis, we often perform **tabular analysis** (as we did in the previous section) for organizing and summarizing the data. However, tabular analysis may fall short in the ability to capture complex trends, patterns, or relationships within the data. This is especially true when dealing with large datasets where patterns may be more difficult to interpret by solely examining the rows of a DataFrame. **Data visualizations** address these limitations by transforming tabular data into intuitive visual displays that leverage human perception to identify patterns and trends. 

This portion of the notebook will motivate the importance of visualizations with the use of standard plotting libraries like [`seaborn`](https://seaborn.pydata.org/) and [`matplotlib`](https://matplotlib.org/stable/). 

### 3.1 Histograms

For the purpose of visualization, will be focusing on eight column attributes from the `ces_cc` DataFrame. Run the code cell below to save the subset as a new DataFrame

In [ ]:
# Run this code cell

ces_subset = ces_cc[['CES 4.0 Score', 'PM2.5', 'Asthma', 'Pollution Burden Score', 'Low Birth Weight', 'Education','Poverty', 'Housing Burden']]

Let's take a look at the **distributions** of some of the columns in `ces_subset` to get a better sense of the data. Rememeber that a distribution describes both the set of values that a single variable can take and the frequency of unique values in a single variable. To do so, we will plot a histogram using seaborn's [`.histplot()`](https://seaborn.pydata.org/generated/seaborn.histplot.html) function to observe the distribution of `CES 4.0 Score` across census tracts. Run the code cell below to familiarize yourself with the syntax of the function. Here, we pass an additional parameter, `stat='density'`, to visualize the distribution in terms of density rather than frequency. 

In [ ]:
sns.histplot(ces_subset, x='CES 4.0 Score', stat='density')
plt.title('The Distribution of CES 4.0 Score Across Census Tracts');

Run the following cell:

In [ ]:
from utils import *

display(ui)

**Question 3.1.1:** Create a histogram based on one of the columns, using the widget above and write a couple of sentences describing the shape of this histogram. What do you notice?

*Type your answer here*

**Question 3.1.1:** Write 1-2 sentences describing the distribution of `CES 4.0 score` based on the histogram above.

*YOUR ANSWER HERE...*

**Question 3.1.1:** Plot a histogram showing the distribution of `Asthma` across census tracts using `ces_set`. Be sure to include a title for your plot. 

In [ ]:
# TODO: Write code to plot a histogram of Asthma using seaborn

... 

Describe the distribution in 1-2 sentences.

*YOUR ANSWER HERE...*

### 3.2 Violin Plots

What if we went to compare the distributions of multiple attributes at once? We can use a **violin plot** which shows the distribution of data points after grouping by one (or more) variables. To create a violin plot, we will use seaborn's `.violinplot()` function. For more information, refer to the function [documentation](https://seaborn.pydata.org/generated/seaborn.violinplot.html).

In [ ]:
# Run this code cell

cities = ['Los Angeles', 'San Bernardino', 'East Los Angeles', 'Compton']
ces_cities = ces[['Approximate Location', 'CES 4.0 Score', 'PM2.5', 'Asthma', 'Pollution Burden Score', 'Low Birth Weight', 'Education','Poverty', 'Housing Burden']]
ces_cities = ces_cities[ces_cities['Approximate Location'].isin(cities)]

In [ ]:
sns.violinplot(ces_cities, x='Approximate Location', y='CES 4.0 Score');

**Question 3.2.1:** Write down 2-3 observations based on the violinplot above.

*YOUR ANSWER HERE...*

### 3.3 Correlations

For the final part of the visualization section, we will briefly discuss how visualizations can help our understanding of relationships between column attributes in our data. 

First, we will begin with pairplots in seaborn. Pairplots allow us to visualize pairwise relationships of column attributes in our data. To create a pairplot, we will call [`.pairplot()`](https://seaborn.pydata.org/generated/seaborn.pairplot.html) and pass in a DataFrame. This will create an $n x n$ grid of plots where n is the number of columns in our data. The histograms along the diagonal of a pairplot describe the distributions of each column attribute. The remaining scatterplots depict the relationship between two columns in the data. Run the following code cell to generate a pairplot. 

In [ ]:
sns.pairplot(ces_subset);

**Question 3.3.1:** From this plot we can see that `Poverty` and `Housing Burden` seem to have a positive, linear relationship which is indicated the scatter of points that fall along a line with a positive slope. What other relationships do you see in the pairplot?

*YOUR ANSWER HERE...*

We can also draw some simple, but powerful analyses by observing the correlation between variables in our data. The code cell below computes the correlation coefficient between various socioeconomic variables in our data and plots a **heatmap** of these coefficients. A heatmap is a visualization that represents the magnitude of values using color. It allows us to easily observe the magnitudes of correlation across pairs of variables.

Run the code cell below and interpret the heatmap.

In [ ]:
# Create a correlation matrix
corr_matrix = ces_subset.corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Between Population Characteristics and Pollution Burden')
plt.show()

**Question 3.3.2:** Which variables are strongly correlated with each other, in either the positive or negative direction? Why might these variables be strongly correlated with one another?

*YOUR ANSWER HERE...*

### 3.4 (Optional) Geopandas Section

CalEnviroScreen is also a mapping tool that allows us to visualize the socioeconomic disparities across California. We will be using a package called `geopandas` to facilitate this mapping. 

The following code cell reads in a shape file (.shp) and contains information that will allow us to map the data using geopandas. A shapefile is a popular geospatial vector data format that stores the geometry of the features in our data. They are common for handling geospatial data. Simply run the cell below to read in the shapefile associated with CalEnviroScreen.

In [ ]:
# Run this code cell

file_path = 'calenviroscreen40shpf2021shp/CES4 Final Shapefile.shp'
gdf = gpd.read_file(file_path)
gdf.head()

We can call `.plot()` to map the data.

In [ ]:
# map the data
gdf.plot()

**Question 3.4.1:** Like we did with the `ces` and `colleges` DataFrames, let's merge the gdf data with collegecodes_public. Following the format from before, merge the necessary datasets on zipcode. 

In [ ]:
# TODO: Fill in the ellipses

enviro_cc_shp = pd.merge(..., ..., how=..., left_on=..., right_on=...)
enviro_cc_shp.head()

We can use geopandas to visualize the spread of our metrics, like `Pollution Burden`. Run the following code cell below. The left plot displays the spread of pollution burden score across California and the right plot displays the spread of poverty across California.  

In [ ]:
# Create a figure with subplots to compare pollution burden and population vulnerability
fig, ax = plt.subplots(1, 2, figsize=(15, 7))

# Plot Pollution Burden on the left
gdf.plot(column='PolBurdSc', ax=ax[0], legend=True, cmap='OrRd')
ax[0].set_title('Pollution Burden Score')

# Plot Population Vulnerability on the right (example with 'Poverty' column)
gdf.plot(column='Poverty', ax=ax[1], legend=True, cmap='PuBu')
ax[1].set_title('Population Vulnerability (Poverty)')

plt.show()

**Question:** List an observation from each plot and how it relates to environmental justice.  

*YOUR ANSWER HERE...*

---

## 4. Data-driven Decision Making

Throughout this notebook, you gained valuable information about how to manipulate data to draw powerful analyses about socioeconomic disparaties across Los Angeles and California. Tools like CalEnviroScreen are essential for evaluating environmnetal justice because they help identify and prioritize communities that face higher environmnetal and health risks due to disproportionate effects of pollution. 

**Question 4.1:** Can you think of how an analysis of two other variables in the data not used in this notebook may be helpful for identifying marginalized communities? You may use the optional code cell as scratch work.

*YOUR ANSWER HERE...*

In [ ]:
# Optional code cell for scratch work

**Question 4.2:** In 3-4 sentences, how might access to CalEnviroScreen data help guide resource allocation and inform policymaking? How might different stakeholders (e.g., local governments, advocacy groups, researchers) use CalEnviroScreen data differently? Give examples of specific decisions or actions they might take.

*YOUR ANSWER HERE...*

**Question 4.3:** Finally, when working with data it is important to address the fundamental concept of **bias**. What potential biases might exist in the CalEnviroScreen data collection or analysis process? How could these biases impact the conclusions drawn from the data?

*YOUR ANSWER HERE...*

---

# Congratulations, you are finished with the notebook!

---

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

These are some submission instructions.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)